In [28]:
## try image cross correlation with stretch

import numpy as np
import matplotlib.pyplot as plt

from skimage import data
from skimage.registration import phase_cross_correlation
from skimage.registration._phase_cross_correlation import _upsampled_dft
from scipy.ndimage import fourier_shift

image = data.camera()
shift = (-22.4, 13.32)
# The shift corresponds to the pixel offset relative to the reference image
offset_image = fourier_shift(np.fft.fftn(image), shift)
offset_image = np.fft.ifftn(offset_image)
print(f'Known offset (y, x): {shift}')

# pixel precision first
shift, error, diffphase = phase_cross_correlation(image, offset_image)

fig = plt.figure(figsize=(8, 3))
ax1 = plt.subplot(1, 3, 1)
ax2 = plt.subplot(1, 3, 2, sharex=ax1, sharey=ax1)
ax3 = plt.subplot(1, 3, 3)

ax1.imshow(image, cmap='gray')
ax1.set_axis_off()
ax1.set_title('Reference image')

ax2.imshow(offset_image.real, cmap='gray')
ax2.set_axis_off()
ax2.set_title('Offset image')

# Show the output of a cross-correlation to show what the algorithm is
# doing behind the scenes
image_product = np.fft.fft2(image) * np.fft.fft2(offset_image).conj()
cc_image = np.fft.fftshift(np.fft.ifft2(image_product))
ax3.imshow(cc_image.real)
ax3.set_axis_off()
ax3.set_title("Cross-correlation")

plt.show()

print(f'Detected pixel offset (y, x): {shift}')

# subpixel precision
shift, error, diffphase = phase_cross_correlation(image, offset_image,
                                                  upsample_factor=100)

fig = plt.figure(figsize=(8, 3))
ax1 = plt.subplot(1, 3, 1)
ax2 = plt.subplot(1, 3, 2, sharex=ax1, sharey=ax1)
ax3 = plt.subplot(1, 3, 3)

ax1.imshow(image, cmap='gray')
ax1.set_axis_off()
ax1.set_title('Reference image')

ax2.imshow(offset_image.real, cmap='gray')
ax2.set_axis_off()
ax2.set_title('Offset image')

# Calculate the upsampled DFT, again to show what the algorithm is doing
# behind the scenes.  Constants correspond to calculated values in routine.
# See source code for details.
cc_image = _upsampled_dft(image_product, 150, 100, (shift*100)+75).conj()
ax3.imshow(cc_image.real)
ax3.set_axis_off()
ax3.set_title("Supersampled XC sub-area")


plt.show()

print(f'Detected subpixel offset (y, x): {shift}')

Known offset (y, x): (-22.4, 13.32)
Detected pixel offset (y, x): [ 22. -13.]
Detected subpixel offset (y, x): [ 22.4  -13.32]


In [29]:
# function for speckle noise
# from https://stackoverflow.com/questions/22937589/how-to-add-noise-gaussian-salt-and-pepper-etc-to-image-in-python-with-opencv


# Parameters
# ----------
# image : ndarray
#     Input image data. Will be converted to float.
# mode : str
#     One of the following strings, selecting the type of noise to add:

#     'gauss'     Gaussian-distributed additive noise.
#     'poisson'   Poisson-distributed noise generated from the data.
#     's&p'       Replaces random pixels with 0 or 1.
#     'speckle'   Multiplicative noise using out = image + n*image,where
#                 n is uniform noise with specified mean & variance.


import numpy as np
import os
import cv2
def noisy(noise_typ,image,weight):
   if noise_typ == "gauss":
      row,col = image.shape
      mean = 0
      gauss = np.random.normal(mean,weight,(row,col))
      gauss = gauss.reshape(row,col)
      noisy = image + gauss
      return noisy
   elif noise_typ == "s&p":
      row,col = image.shape
      s_vs_p = weight
      amount = 0.004
      out = np.copy(image)
      # Salt mode
      num_salt = np.ceil(amount * image.size * s_vs_p)
      coords = [np.random.randint(0, i - 1, int(num_salt))
              for i in image.shape]
      out[coords] = 1

      # Pepper mode
      num_pepper = np.ceil(amount* image.size * (1. - s_vs_p))
      coords = [np.random.randint(0, i - 1, int(num_pepper))
              for i in image.shape]
      out[coords] = 0
      return out
   elif noise_typ == "poisson":
      vals = len(np.unique(image))
      vals = 2 ** np.ceil(np.log2(vals))
      noisy = np.random.poisson(image * vals) / float(vals)
      return noisy
   elif noise_typ =="speckle":
      row,col = image.shape
      gauss = weight * np.random.randn(row,col)
      gauss = gauss.reshape(row,col)        
      noisy = image + image * gauss
      noisy = image * gauss
      return noisy
   
   elif noise_typ == 'chi_speckle':
      row,col = image.shape
      chi = weight * np.random.chisquare(2,
                                         (row,col))
      # noisy = image + image * chi
      noisy = image * chi
      return noisy
   elif noise_typ == 'gamma':
      row,col = image.shape
      N = weight * np.random.gamma(1,1,(row,col))
      # noisy = image + image * chi
      noisy = image * N
      return noisy

In [30]:
## try image cross correlation with stretch

import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import cv2
from skimage import data
from skimage.registration import phase_cross_correlation
from skimage.registration._phase_cross_correlation import _upsampled_dft
from scipy.ndimage import fourier_shift
from skimage.feature import match_template



In [38]:
## load sim sar
import matplotlib.colors as mcolors
scale= 1
expo = 0.35
r_win = 388
a_win = 188
region = [6150,6150+a_win,3700,3700+r_win]
sim_sar = np.fromfile('../SPOTSAR/test_data/CSK_dsc/misc/MLIs/sim_sar.rdc', dtype='>f', count=-1)
true_sar = np.fromfile('../SPOTSAR/test_data/CSK_dsc/misc/MLIs/c20200910_full.rmli', dtype='>f', count=-1)

# sim_sar = sim_sar**(1/expo)
# sim_sar = scale*np.exp(exp*sim_sar)
# true_sar = scale*(true_sar**expo)
width = 17478
lines = int(sim_sar.shape[0]/width)

sim_sar = np.reshape(sim_sar,[lines,width])[region[0]:region[1],region[2]:region[3]]
true_sar = np.reshape(true_sar,[lines,width])[region[0]:region[1],region[2]:region[3]]

fig,ax = plt.subplots(1,2)

ax[0].imshow(sim_sar,cmap='Greys_r')
# ,vmin=0,vmax=np.max(true_sar))
ax[1].imshow(true_sar,cmap='Greys_r',vmin=0,vmax=np.max(true_sar))


In [39]:
## create noisy images
weight = 1
scale = 1
sim_sar_scale = sim_sar**(1/expo)
sim_sar_scale = sim_sar
noisy1 = scale*noisy('chi_speckle',sim_sar_scale,weight)
noisy2 = scale*noisy('chi_speckle',sim_sar_scale,weight)
noisy1 = scale*noisy('gamma',sim_sar_scale,weight)
noisy2 = scale*noisy('gamma',sim_sar_scale,weight)

# noisy1 = noisy('chi_speckle',true_sar,weight)
# noisy2 = noisy('chi_speckle',true_sar,weight)

fig, ax = plt.subplots(1,4)
cc_plot0 = ax[0].imshow(scale*sim_sar_scale, norm=mcolors.LogNorm(vmin=1, vmax=1e7))
plt.colorbar(cc_plot0)
cc_plot1 = ax[1].imshow(noisy1, norm=mcolors.LogNorm(vmin=1, vmax=1e7))
plt.colorbar(cc_plot1)
cc_plot2 = ax[2].imshow(noisy2, norm=mcolors.LogNorm(vmin=1, vmax=1e7))
plt.colorbar(cc_plot2)
cc_plot3 = ax[3].imshow(true_sar, norm=mcolors.LogNorm(vmin=1, vmax=1e7))
plt.colorbar(cc_plot3)

In [40]:


image_shape = np.shape(noisy1)
print(image_shape)





(188, 388)


In [41]:
print(np.shape(noisy1))
print(np.shape(noisy2))
stretch_vec = np.linspace(1.01,3,99)
print(stretch_vec)
ccp = np.empty((np.size(stretch_vec),2))
for i, stretch in enumerate(stretch_vec):
    print(i,stretch)
    # The shift corresponds to the pixel offset relative to the reference image

    resized_image = cv2.resize(noisy2,(int(np.round(image_shape[1]*stretch)),image_shape[0]))
    # resized_image = cv2.resize(noisy_image,(int(np.round(image_shape[0]*stretch)),int(np.round(image_shape[1]*stretch))))
    print(np.shape(resized_image))
    print(np.shape(noisy1),np.shape(resized_image))
    # pixel precision first
    # shift, error, diffphase = phase_cross_correlation(image, resized_image)
    if stretch<=1:
        result = match_template(noisy1,resized_image)
    else:
        result = match_template(resized_image,noisy1)

    # fig, ax = plt.subplots(1,1)
    # cc_plot = ax.imshow(image)
    # plt.colorbar(cc_plot)
    # fig, ax = plt.subplots(1,1)
    # cc_plot = ax.imshow(resized_image)
    # plt.colorbar(cc_plot)
    # fig, ax = plt.subplots(1,1)
    # cc_plot = ax.imshow(result)
    # plt.colorbar(cc_plot)
    ccp[i,:] = [stretch, np.max(result)]
    # ccp = np.max(result)


(188, 388)
(188, 388)
[1.01       1.03030612 1.05061224 1.07091837 1.09122449 1.11153061
 1.13183673 1.15214286 1.17244898 1.1927551  1.21306122 1.23336735
 1.25367347 1.27397959 1.29428571 1.31459184 1.33489796 1.35520408
 1.3755102  1.39581633 1.41612245 1.43642857 1.45673469 1.47704082
 1.49734694 1.51765306 1.53795918 1.55826531 1.57857143 1.59887755
 1.61918367 1.6394898  1.65979592 1.68010204 1.70040816 1.72071429
 1.74102041 1.76132653 1.78163265 1.80193878 1.8222449  1.84255102
 1.86285714 1.88316327 1.90346939 1.92377551 1.94408163 1.96438776
 1.98469388 2.005      2.02530612 2.04561224 2.06591837 2.08622449
 2.10653061 2.12683673 2.14714286 2.16744898 2.1877551  2.20806122
 2.22836735 2.24867347 2.26897959 2.28928571 2.30959184 2.32989796
 2.35020408 2.3705102  2.39081633 2.41112245 2.43142857 2.45173469
 2.47204082 2.49234694 2.51265306 2.53295918 2.55326531 2.57357143
 2.59387755 2.61418367 2.6344898  2.65479592 2.67510204 2.69540816
 2.71571429 2.73602041 2.75632653 2.7766

In [42]:
%matplotlib osx
fig, ax = plt.subplots(1,1)
cc_plot = ax.imshow(noisy1)
plt.colorbar(cc_plot)
fig, ax = plt.subplots(1,1)
cc_plot = ax.imshow(noisy2)
plt.colorbar(cc_plot)

print(ccp)
fig, ax = plt.subplots(1,1)
ax.plot(ccp[:,0],ccp[:,1]
        )
ax.set_xlabel('stretch 1D [-]')
ax.set_ylabel('cross-corr. peak [-]')

[[1.01       0.29095484]
 [1.03030612 0.22302979]
 [1.05061224 0.19926808]
 [1.07091837 0.19356912]
 [1.09122449 0.18156579]
 [1.11153061 0.17431851]
 [1.13183673 0.17014728]
 [1.15214286 0.164905  ]
 [1.17244898 0.15971039]
 [1.1927551  0.16235145]
 [1.21306122 0.15753305]
 [1.23336735 0.15250808]
 [1.25367347 0.14605613]
 [1.27397959 0.14334553]
 [1.29428571 0.15039304]
 [1.31459184 0.14796103]
 [1.33489796 0.14614499]
 [1.35520408 0.14428521]
 [1.3755102  0.14083363]
 [1.39581633 0.14695741]
 [1.41612245 0.14616954]
 [1.43642857 0.14205735]
 [1.45673469 0.13917142]
 [1.47704082 0.1400404 ]
 [1.49734694 0.13881919]
 [1.51765306 0.13332027]
 [1.53795918 0.13417083]
 [1.55826531 0.13524848]
 [1.57857143 0.13447247]
 [1.59887755 0.13381352]
 [1.61918367 0.13157209]
 [1.6394898  0.13243851]
 [1.65979592 0.13330776]
 [1.68010204 0.128734  ]
 [1.70040816 0.12783981]
 [1.72071429 0.12853734]
 [1.74102041 0.12994503]
 [1.76132653 0.12930292]
 [1.78163265 0.12608291]
 [1.80193878 0.12351288]


Text(0, 0.5, 'cross-corr. peak [-]')

1   HIToolbox                           0x00007ff817e57726 _ZN15MenuBarInstance22EnsureAutoShowObserverEv + 102
2   HIToolbox                           0x00007ff817e20638 _ZL17BroadcastInternaljPvh + 167
3   SkyLight                            0x00007ff8130c523d _ZN12_GLOBAL__N_123notify_datagram_handlerEj15CGSDatagramTypePvmS1_ + 1030
4   SkyLight                            0x00007ff8133d805a _ZN21CGSDatagramReadStream26dispatchMainQueueDatagramsEv + 202
5   SkyLight                            0x00007ff8133d7f81 ___ZN21CGSDatagramReadStream15mainQueueWakeupEv_block_invoke + 18
6   libdispatch.dylib                   0x00007ff80e23c7fb _dispatch_call_block_and_release + 12
7   libdispatch.dylib                   0x00007ff80e23da44 _dispatch_client_callout + 8
8   libdispatch.dylib                   0x00007ff80e24a7b9 _dispatch_main_queue_drain + 952
9   libdispatch.dylib                   0x00007ff80e24a3f3 _dispatch_main_queue_callback_4CF + 31
10  CoreFoundation                      

In [63]:
scale= 1
expo = 0.35
a_wins = [100,200,300,400,500]
r_wins = [100,200,300,400,500]
stretch_vec = np.linspace(1.01,1.5,100)
print(stretch_vec)
sim_sar = np.fromfile('../SPOTSAR/test_data/CSK_dsc/misc/MLIs/sim_sar.rdc', dtype='>f', count=-1)
# true_sar = np.fromfile('../SPOTSAR/test_data/CSK_dsc/misc/MLIs/c20200910_full.rmli', dtype='>f', count=-1)

width = 17478
lines = int(sim_sar.shape[0]/width)

fig,ax = plt.subplots(1,1)

for (r_win,a_win) in zip(r_wins,a_wins):
    region = [700,700+a_win,5500,5500+r_win]
    sim_sar2 = np.reshape(sim_sar,[lines,width])[region[0]:region[1],region[2]:region[3]]
    # true_sar = np.reshape(true_sar,[lines,width])[region[0]:region[1],region[2]:region[3]]

    ## create noisy images
    weight = 1
    scale = 1

    # noisy1 = scale*noisy('chi_speckle',sim_sar,weight)
    # noisy2 = scale*noisy('chi_speckle',sim_sar,weight)
    noisy1 = scale*noisy('gamma',sim_sar2,weight)
    noisy2 = scale*noisy('gamma',sim_sar2,weight)

    fig1,ax1 = plt.subplots(1,3)
    ax1[0].imshow(sim_sar2)
    ax1[1].imshow(noisy1)
    ax1[2].imshow(noisy2)



    ccp = np.empty((np.size(stretch_vec),2))
    for i, stretch in enumerate(stretch_vec):
        print(i,stretch)
        # The shift corresponds to the pixel offset relative to the reference image

        resized_image = cv2.resize(noisy2,(int(np.round(np.shape(noisy2)[0]*stretch)),np.shape(noisy2)[1]))
        # resized_image = cv2.resize(noisy_image,(int(np.round(image_shape[0]*stretch)),int(np.round(image_shape[1]*stretch))))
        print(np.shape(noisy1),np.shape(noisy2),np.shape(resized_image))
        # pixel precision first
        # shift, error, diffphase = phase_cross_correlation(image, resized_image)
        if stretch<=1:
            result = match_template(noisy1,resized_image)
        else:
            result = match_template(resized_image,noisy1)


        ccp[i,:] = [stretch, np.max(result)]
    ax.plot(ccp[:,0]-1,ccp[:,1],label=f'window size: {r_win}')
ax.legend()
ax.set_xlabel('Strain [-]')
ax.set_ylabel('Peak normalised cross-correlation [-]')



[1.01       1.01494949 1.01989899 1.02484848 1.02979798 1.03474747
 1.03969697 1.04464646 1.04959596 1.05454545 1.05949495 1.06444444
 1.06939394 1.07434343 1.07929293 1.08424242 1.08919192 1.09414141
 1.09909091 1.1040404  1.1089899  1.11393939 1.11888889 1.12383838
 1.12878788 1.13373737 1.13868687 1.14363636 1.14858586 1.15353535
 1.15848485 1.16343434 1.16838384 1.17333333 1.17828283 1.18323232
 1.18818182 1.19313131 1.19808081 1.2030303  1.2079798  1.21292929
 1.21787879 1.22282828 1.22777778 1.23272727 1.23767677 1.24262626
 1.24757576 1.25252525 1.25747475 1.26242424 1.26737374 1.27232323
 1.27727273 1.28222222 1.28717172 1.29212121 1.29707071 1.3020202
 1.3069697  1.31191919 1.31686869 1.32181818 1.32676768 1.33171717
 1.33666667 1.34161616 1.34656566 1.35151515 1.35646465 1.36141414
 1.36636364 1.37131313 1.37626263 1.38121212 1.38616162 1.39111111
 1.39606061 1.4010101  1.4059596  1.41090909 1.41585859 1.42080808
 1.42575758 1.43070707 1.43565657 1.44060606 1.44555556 1.45050

Text(0, 0.5, 'Peak normalised cross-correlation [-]')